In [1]:
import pandas as pd
import numpy as np

import re
from itertools import chain
from collections import Counter

from google.cloud import bigquery
from google.oauth2 import service_account

from datetime import datetime, timedelta

import requests
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning

# InsecureRequestWarning 경고 무시
warnings.simplefilter('ignore', InsecureRequestWarning)

In [2]:
KEY_PATH = ".config/"
servicekey_path = KEY_PATH + "serviceKey.json" ## 빅쿼리 외 다른 API 활용 위해
bigquerykey_path = KEY_PATH + "mido-project-426906-31b49963ac97.json"

warnings.filterwarnings("ignore")

In [3]:
# BigQuery 클라이언트 생성 함수
def create_bigquery_client(key_path):
    credentials = service_account.Credentials.from_service_account_file(key_path)
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)
    return client

In [4]:
def save_dataframe_to_bigquery(df, dataset_id, table_id, key_path):
    # BigQuery 클라이언트 객체 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 데이터프레임을 BigQuery 테이블에 적재
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = "WRITE_TRUNCATE"  # 기존 테이블 내용 삭제 후 삽입

    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()  # 작업 완료 대기

    print(f"Data inserted into table {table_id} successfully.")

In [5]:
def get_dataframe_from_bigquery(dataset_id, table_id, key_path):
    # BigQuery 클라이언트 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 테이블 데이터를 DataFrame으로 변환
    df = client.list_rows(table_ref).to_dataframe()

    return df

In [6]:
# 오늘 날짜
today = datetime.today()#.strftime('%Y%m%d')

# 어제 날짜 계산
ytday = datetime.today() - timedelta(days=1)

# 만약 어제, 오늘이 토요일(5) 또는 일요일(6)이라면, 그 전주 금요일로 변경
if ytday.weekday() == 5:  # 토요일
    ytday -= timedelta(days=1)
elif ytday.weekday() == 6:  # 일요일
    ytday -= timedelta(days=2)
if today.weekday() == 5:  # 토요일
    today -= timedelta(days=1)
elif today.weekday() == 6:  # 일요일
    today -= timedelta(days=2)

# 'YYYYMMDD' 형식으로 변환
ytday = ytday.strftime('%Y%m%d')
today = today.strftime('%Y%m%d')

#### 법정동코드

In [7]:
dist_code = pd.read_csv('C:\py_src\midoproject\data/법정동코드 전체자료.csv',encoding='cp949')

In [8]:
dist_code_list = []
for i in range(dist_code['법정동명'].str.split().apply(len).max()):
    dist_code_temp = dist_code[dist_code['법정동명'].str.split().str[i].str[-1].isin(['읍','면','동','리','가','로']).dropna()]
    dist_code_list.append([dist_code_temp['법정동명'].str.split().str[i].dropna().unique()])

emd_code_list = list(chain(*chain(*dist_code_list)))

#### 종합쇼핑몰 납품상세내역

In [107]:
all_shop_df = get_dataframe_from_bigquery('g2b', 'shop_detail_df_total', bigquerykey_path)

In [111]:
all_shop_df['우수제품여부'].unique()

array(['Y', None, 'N'], dtype=object)

In [71]:
# years = today[:-4]
# all_shop_df = all_shop_df[all_shop_df['납품요구접수일자'].str.split('-').str[0]==years].reset_index(drop=True)
# all_shop_df_fin = all_shop_df[['납품요구접수일자', '수요기관명', '납품요구건명', '업체명', '단가', '단위', '수량', '금액', '품목, '업체명,'수요기관지역명','납품요구지청명']]
# all_shop_df['납품요구접수일자'] = pd.to_datetime(all_shop_df['납품요구접수일자'], errors='coerce')

In [72]:
all_shop_df['납품요구변경차수'] = all_shop_df['납품요구변경차수'].astype(int)
all_shop_df = all_shop_df.loc[all_shop_df.groupby(['납품요구번호', '물품순번'])['납품요구변경차수'].idxmax()]
all_shop_df = all_shop_df.sort_values(['납품요구접수일자'],ascending=False).reset_index(drop=True)

In [73]:
all_shop_df_fin = all_shop_df[['납품요구접수일자', '수요기관명', '납품요구건명', '업체명', '단가', '단위', '수량', '금액', '품목', '수요기관지역명', '납품요구지청명','업체기업구분명','우수제품여부']]

In [74]:
# 컬럼명을 한글로 변경
column_mapping = {
    '납품요구접수일자' : 'dlvrReqRcptDate',
    '품목' : 'prdctIdntNoNm',
    '단가' : 'prdctUprc',
    '단위' : 'prdctUnit',
    '수량' : 'prdctQty',
    '금액' : 'prdctAmt',
    '우수제품여부' : 'exclcProdctYn',
    '수요기관명' : 'dminsttNm',
    '수요기관지역명' : 'dminsttRgnNm',
    '업체명' : 'corpNm',
    '납품요구건명' : 'dlvrReqNm',
    '업체기업구분명' : 'corpEntrprsDivNmNm',
    '납품요구지청명' : 'brnofceNm'
}

In [75]:
all_shop_df_fin.rename(columns=column_mapping, inplace=True)

In [77]:
# 빅쿼리 적재
save_dataframe_to_bigquery(all_shop_df_fin,'g2b','shop_detail_df_view',bigquerykey_path)

Data inserted into table shop_detail_df_view successfully.
